In [1]:
import numpy as np
import cv2

In [2]:
with_mask=np.load('with Mask.npy')

In [3]:
with_mask.shape

(200, 50, 50, 3)

In [4]:
without_mask=np.load('without Mask.npy')

In [5]:
without_mask.shape

(200, 50, 50, 3)

In [6]:
without_mask=without_mask.reshape(200,50*50*3)

In [7]:
with_mask=with_mask.reshape(200,50*50*3)

In [8]:
with_mask.shape

(200, 7500)

In [9]:
without_mask.shape

(200, 7500)

In [10]:
x=np.r_[with_mask,without_mask]

In [11]:
names={0:'Mask',1:'No Mask'}

In [12]:
x.shape

(400, 7500)

In [13]:
labels=np.zeros(x.shape[0])

In [14]:
labels[200:] = 1.0

In [15]:
labels

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [16]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [17]:
x_train,x_test,y_train,y_test = train_test_split(x,labels,test_size=0.35)

In [18]:
x_train.shape

(260, 7500)

In [19]:
from sklearn.decomposition import PCA   # for dimensionality reduction

In [20]:
pca=PCA(n_components=3)

In [21]:
x_train=pca.fit_transform(x_train)

In [22]:
x_train[0]

array([  54.59446671, 1693.41918526,  925.40789809])

In [23]:
x_train.shape

(260, 3)

In [24]:
model=SVC()
model.fit(x_train,y_train)

SVC()

In [25]:
x_test=pca.fit_transform(x_test)
predt=model.predict(x_test)

In [26]:
predt

array([0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1.,
       1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
       0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 1.])

In [27]:
accuracy_score(y_test,predt)

1.0

In [28]:
# capturing images from default camera
df=cv2.CascadeClassifier('haar_data.xml')
capture = cv2.VideoCapture(0) # 0 for default camera 1 for another installed camera
font=cv2.FONT_HERSHEY_COMPLEX
while True:
    
        flag,image = capture.read() # if flag is working its gonn detect the face
        
        if flag: 
            faces = df.detectMultiScale(image)
            
            for x,y,w,h in faces:
                
                cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,255),4)
                face=image[y:y+h,x:x+w,:]
                face=cv2.resize(face,(50,50))
                face=face.reshape(1,-1)
                face=pca.transform(face)
                predt=model.predict(face)
                
                n=names[int(predt)]
                cv2.putText(image,n,(x,y),font ,1,(250,250,250),2)   # 1 shows the 16 pixel image  and 2 for weight
                print(n) 
                                
            cv2.imshow('Result',image)
                # 27 - ASCII of escape
            if cv2.waitKey(2) == 27 :
                    break 
                    
capture.release()
cv2.destroyAllWindows()

Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
No Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
No Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
No Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
No Mask
Mask
Mask
Mask
No Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
No Mask
Mask
Mask
Mask
No Mask
Mask
Mask
No Mask
Mask
Mask
No Mask
Mask
Mask
Mask
No Mask
No Mask
Mask
Mask
Mask
Mask
No Mask
Mask
No Mask
Mask
No Mask
Mask
Mask
Mask
No Mask
Mask
Mask
